In [126]:
%matplotlib widget
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import pickle
import scmdata
import xarray as xr
from itertools import product
from cftime import DatetimeGregorian

In [108]:
def scmrun_from_pandas(df, region, variable, scenario, model):
    columns = list(df.index.names)
    columns.remove('Date')
    columns.append('Date')
    dfs = df.pint.magnitude.reorder_levels(columns).sort_index().unstack()
    
    def get_level_values_from_spec(spec):
        if isinstance(spec, str):
            return dfs.index.get_level_values(spec)
        elif callable(spec):
            return spec(dfs)
        else:
            return spec
    
    return scmdata.ScmRun(dfs.values.T,
                          columns={'region': get_level_values_from_spec(region),
                                   'variable': get_level_values_from_spec(variable),
                                   'unit': [str(df.pint.units)],
                                   'scenario': get_level_values_from_spec(scenario),
                                   'model': get_level_values_from_spec(model)},
                          index=dfs.columns)

In [109]:
with open(f'mem/fao_emi_total.pck', 'rb') as fd:
    fao_emi_total_pd = pickle.load(fd)

def variable(dfs):
    elements = pd.Series(dfs.index.get_level_values('Element')).astype(pd.StringDtype())
    items = pd.Series(dfs.index.get_level_values('Item')).astype(pd.StringDtype())
    return elements + ' - ' + items

fao_emi_total = scmrun_from_pandas(fao_emi_total_pd,
                                   region='Area',
                                   variable=variable,
                                   scenario=['history'],
                                   model=['FAO'])

In [112]:
with open(f'mem/lak_emi_energy.pck', 'rb') as fd:
    lak_emi_energy_pd = pickle.load(fd)

lak_emi_energy = scmrun_from_pandas(lak_emi_energy_pd,
                                    region='Area',
                                    variable='Category',
                                    scenario=['history'],
                                    model=['LAK'])

In [157]:
with open(f'mem/cmip_conc_co2.pck', 'rb') as fd:
    cmip_conc_co2_pd = pickle.load(fd)

cmip_conc_co2_pd.index.set_levels(
    [f'{year:04}-{month:02}-01' for year, month in product([-1] + list(range(1, 2015)), range(1, 13))],
    level='Date',
    inplace=True)    

cmip_conc_co2 = scmrun_from_pandas(cmip_conc_co2_pd,
                                   region='Area',
                                   variable=['CO2 concentration'],
                                   scenario=['history'],
                                   model=['CMIP'])

In [161]:
lak_emi_energy

<scmdata.ScmRun (timeseries: 112, timepoints: 29)>
Time:
	Start: 1990-01-01T00:00:00
	End: 2018-01-01T00:00:00
Meta:
	                  region                           variable            unit  \
	63628  Baden-Württemberg             Abfälle (nicht biogen)  kilometric_ton   
	63629  Baden-Württemberg               Andere Energieträger  kilometric_ton   
	63630  Baden-Württemberg                         Braunkohle  kilometric_ton   
	63631  Baden-Württemberg                               Gase  kilometric_ton   
	63632  Baden-Württemberg                          Insgesamt  kilometric_ton   
	...                  ...                                ...             ...   
	63735          Thüringen                         Braunkohle  kilometric_ton   
	63736          Thüringen                               Gase  kilometric_ton   
	63737          Thüringen                          Insgesamt  kilometric_ton   
	63738          Thüringen  Mineralöle und Mineralöl produkte  kilometric_ton   
	63